# Nearest neighbours benchmarks for PR10212

*Roman Yurchak*, November 29 2017

(adapted from the code proposed by Loic Esteve)

Requires scikit-learn, pandas, hypothesis and tqdm

## Setup

In [1]:
import timeit
from functools import partial

import numpy as np
import pandas as pd
import scipy.sparse

from sklearn.utils.extmath import row_norms
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

import hypothesis.strategies as st

metric = 'euclidean'

rng = np.random.RandomState(42)

def fast_euclidean_neighbors(X, X_queries, X_norms, n_queries, n_neighbors):
    """Quick function extracting bits of pieces from the NearestNeighbors code"""
    norms_squared = np.dot(X_queries, X.T)
    norms_squared *= -2
    norms_squared += row_norms(X_queries, squared=True)[:, np.newaxis]
    norms_squared += X_norms
    indices = np.argpartition(norms_squared, n_neighbors - 1, axis=1)
    indices = indices[:, :n_neighbors]

    neighbors_range = np.arange(n_queries)[:, np.newaxis]
    norms_squared = norms_squared[neighbors_range, indices]
    arg_ind = np.argsort(norms_squared, axis=1)
    norms = np.sqrt(norms_squared[neighbors_range, arg_ind])
    indices = indices[neighbors_range, arg_ind]
    return norms, indices


def benchmark_nn(n_samples, n_features, n_queries, n_neighbors, sparse):
    results = {'n_samples': n_samples, 'n_features': n_features,
               'n_queries': n_queries, 'n_neighbors': n_neighbors,
               'sparse': int(sparse)}   
    if sparse:
        X = scipy.sparse.rand(n_samples, n_features, random_state=rng)
        X_queries = scipy.sparse.rand(n_queries, n_features, random_state=rng)
    else:
        X = rng.randn(n_samples, n_features)
        X_queries = rng.randn(n_queries, n_features)
    
    X_norms = row_norms(X, squared=True)

    nn = NearestNeighbors(algorithm='brute', metric=metric)
    nn.fit(X)
    
    t_master_fit = timeit.repeat(partial(nn.fit, X),
                                 number=1, repeat=3)
    t_master_kneighbors = timeit.repeat(partial(nn.kneighbors, X_queries, n_neighbors=n_neighbors),
                                        number=1, repeat=3)
    results['time_master_fit'] = np.mean(t_master_fit)
    results['time_master_kneighbors'] = np.mean(t_master_kneighbors)
    
    t_PR_kneighbors = timeit.repeat(partial(fast_euclidean_neighbors, X, X_queries, X_norms, n_queries, n_neighbors),
                                    number=1, repeat=3)
    results['time_PR_kneighbors'] = np.mean(t_PR_kneighbors)
    t_PR_fit = timeit.repeat(partial(row_norms, X, squared=True),
                             number=1, repeat=3)
    # this PR roughly ads the row_norms operation to fit runtime
    results['time_PR_fit'] = np.mean(t_master_fit) + np.mean(t_PR_fit)
    results['speedup_fit'] = results['time_master_fit'] / results['time_PR_fit']
    results['speedup_kneighbors'] = results['time_master_kneighbors'] / results['time_PR_kneighbors']
    results['speedup_fit_kneighbors'] = (results['time_master_fit'] + results['time_master_kneighbors']) \
                                      / (results['time_PR_fit'] + results['time_PR_kneighbors'])
    return results

In [2]:
def prepare_dataframe(df):
    df_s = df.set_index(['n_samples', 'n_features', 'n_neighbors',  'n_queries'])
    df_s = df_s[['speedup_fit', 'speedup_kneighbors', 'speedup_fit_kneighbors']]
    return df_s.sort_values('speedup_fit_kneighbors')

## Results

In these benchmarks we define the parameter space in `n_samples`, `n_features`, `n_queries`, `n_neighbours` for both dense and sparse array, then randomly sample it with hypothesis.

We will consider the speedup (ratio of the run time for the proposed code with that on master) for `speedup_fit` (the `fit` method), `speedup_kneighbours` (the `kneighbours` method) and the `speedup_fit_kneighbours` (for the `fit` + `kneighbours` methods). For instance a speedup of 2. means that the proposed change is twice faster for the given parameters.

### Benchmark results for dense arrays

In [3]:
# generate the sampling strategy with hypothesis

par_space_dense = st.tuples(st.integers(100, 5000), # n_samples
                            st.integers(2, 1000),  # n_features
                            st.integers(1, 100), # n_queries
                            st.integers(2, 10), # n_neighbours
                            st.just(False) # sparse
                            )


df_dense = pd.DataFrame(benchmark_nn(*par_space_dense.example()) for _ in tqdm(range(1000)))
df_dense = prepare_dataframe(df_dense)

100%|██████████| 1000/1000 [01:57<00:00, 11.21it/s]




####  A random sample of the results

In [4]:
df_dense.sample(10)

,,,,speedup_fit,speedup_kneighbors,speedup_fit_kneighbors
n_samples,n_features,n_neighbors,n_queries,,,
1796,455,5,62,0.400500,1.343248,1.152085
4930,526,3,81,0.398116,1.252576,1.104735
2233,339,10,39,0.455592,1.414867,1.186093
423,268,7,38,0.520332,1.369849,1.184493
4720,327,2,72,0.487499,1.204152,1.073292
2074,642,6,89,0.402121,1.186813,1.072162
1372,798,9,21,0.426804,1.690292,1.215654
2932,345,7,2,0.424906,2.554844,1.355044
1801,832,4,39,0.409107,1.443838,1.165650


#### Best cases

In [5]:
df_dense.tail()

,,,,speedup_fit,speedup_kneighbors,speedup_fit_kneighbors
n_samples,n_features,n_neighbors,n_queries,,,
2654,27,2,50,0.627343,2.131957,1.940485
320,34,5,20,0.804502,2.390362,1.956263
3088,3,2,3,0.677750,2.860369,1.982658
4476,29,8,2,0.647438,3.258557,2.047236
387,24,8,12,0.861811,3.939383,2.612529


#### Worst cases

In [6]:
df_dense.head()

,,,,speedup_fit,speedup_kneighbors,speedup_fit_kneighbors
n_samples,n_features,n_neighbors,n_queries,,,
606,690,6,96,0.444366,0.954332,0.891397
634,432,6,46,0.512731,1.042280,0.907282
2697,414,8,86,0.451577,1.010128,0.925370
723,756,6,63,0.448331,1.011875,0.929831
870,85,9,41,0.607824,0.991767,0.936266


### Benchmark results for sparse arrays

In [7]:
# generate the sampling strategy with hypothesis

par_space_sparse =  st.tuples(st.integers(100, 5000), # n_samples
                                st.integers(1000, 10000),  # n_features
                                st.integers(1, 100), # n_queries
                                st.integers(2, 10), # n_neighbours
                                st.just(True) # sparse
                              )

# running the test for dense arrays
df_sparse = pd.DataFrame(benchmark_nn(*par_space_sparse.example()) for _ in tqdm(range(100)))
df_sparse = prepare_dataframe(df_sparse)

100%|██████████| 100/100 [00:55<00:00,  2.70it/s]




####  A random sample of the results

In [11]:
df_sparse.sample(10)

,,,,speedup_fit,speedup_kneighbors,speedup_fit_kneighbors
n_samples,n_features,n_neighbors,n_queries,,,
1785,4779,5,50,0.475857,0.344621,0.364566
3061,9435,7,76,0.467614,0.300838,0.323891
4300,9191,7,3,0.313318,0.089291,0.122452
3321,8502,10,73,0.529920,0.315015,0.347521
3205,2511,6,83,0.445630,0.474351,0.470813
746,9485,6,69,0.502880,0.398730,0.413351
1729,5208,9,11,0.471164,0.215260,0.255335
1102,4794,4,51,0.441795,0.357748,0.369439
4890,1982,2,86,0.355696,0.446752,0.431506


#### Best cases

In [12]:
df_sparse.tail()

,,,,speedup_fit,speedup_kneighbors,speedup_fit_kneighbors
n_samples,n_features,n_neighbors,n_queries,,,
137,7136,8,45,0.449678,0.735553,0.682165
535,2178,6,68,0.452804,0.780020,0.719185
846,1783,7,76,0.531624,0.870250,0.806027
108,3241,5,95,0.349130,1.064493,0.871620
232,1259,4,69,0.660730,1.092807,0.948956


#### Worst cases

In [13]:
df_sparse.head()

,,,,speedup_fit,speedup_kneighbors,speedup_fit_kneighbors
n_samples,n_features,n_neighbors,n_queries,,,
4300,9191,7,3,0.313318,0.089291,0.122452
3324,9492,2,2,0.489204,0.138278,0.195480
4706,9992,3,5,0.459061,0.148413,0.197625
4964,5176,4,15,0.459356,0.177310,0.225949
4877,6625,9,24,0.457511,0.181618,0.226640
